# Pass Through do câmbio na inflação

Neste exercício, vamos seguir a publicação realizada pelo site Análise Macro, em seu Boletim AM, que sempre traz conteúdos de economia aplicada. Para conferir o desenvolvimento original do autor, segue o link:

https://analisemacro.com.br/economia/macroeconometria/estimando-o-repasse-externo-sobre-a-inflacao-brasileira/

Trarei aqui de maneira mais simplificado os temas, pois assim mantenho o papel do post original, não o "copiando" por completo e ainda desenvolvendo o  exercício.

A estimação do produto será realizado pelo *Generalized Method of Moments (GMM)*. As variáveis utilizadas no modelo são:

* Hiato do produto - BCB
* IC-Br USD (Var. % Trim.) - BCB
* Expectativa de 12 meses do IPCA Total (a.a.%) - BCB
* IPCA EX01 (a.a.%) - BCB

#### **1.** Carregando libs


In [8]:
%pip install python-bcb
%pip install linearmodels

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
from bcb import sgs #importando o Sistema Gerenciador de Séries Temporais do BC
from bcb import Expectativas #importanto dados de expectativas 
import pandas as pd #para manipular os dados
import numpy as np #para manipular números e realizar cálculos
from functools import reduce # não sei
import seaborn as sns # biblioteca para visualização de dados estatistícos baseadas em matplotlib
import matplotlib.pyplot as plt # biblioteca para plotar gráficos
from plotnine import * # outra biblioteca de apoio para a construção dos gráficos

#### **2.** Coleta e tratamento de dados

##### **2.1.** Expectativas de inflação

In [10]:
# instancia a classe de expectativas
em = Expectativas()

# obtém o último ponto da expectativa da inflação acumulada em 12 meses
exp_ipca_raw = em.get_endpoint('ExpectativasMercadoInflacao12Meses')

# traz a expectativa de inflação cambial acumulada em 12 meses
exp_ipca = (
    exp_ipca_raw.query() # começa uma nova consulta
    .filter(exp_ipca_raw.Indicador == 'IPCA') # filtra só o IPCA
    .filter(exp_ipca_raw.baseCalculo == '0') # filtra a base de cálculo para 0 <<não entendi>>
    .filter(exp_ipca_raw.Suavizada == 'S') # filtra suavizada para S <<não entendi>>
    .collect() # coleta os dados da query
) 



In [11]:
exp_ipca

,Indicador,Data,Suavizada,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo
0,IPCA,2001-12-12,S,5.1700,5.2200,0.5600,NaN,NaN,NaN,0
1,IPCA,2001-12-13,S,5.1600,5.2200,0.5500,NaN,NaN,NaN,0
2,IPCA,2001-12-14,S,5.1300,5.1500,0.5700,NaN,NaN,NaN,0
3,IPCA,2001-12-17,S,5.1200,5.1600,0.5700,NaN,NaN,NaN,0
4,IPCA,2001-12-18,S,5.0900,5.1300,0.5800,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
5481,IPCA,2023-10-16,S,3.9841,4.0109,0.4064,2.9552,5.4359,130.0,0
5482,IPCA,2023-10-17,S,3.9808,3.9951,0.4043,2.9566,5.4353,131.0,0
5483,IPCA,2023-10-18,S,3.9741,3.9797,0.4033,2.9581,5.4346,132.0,0
5484,IPCA,2023-10-19,S,3.9692,3.9688,0.4007,2.9596,5.4340,132.0,0


In [12]:
# tratamento da IPCA-EX0
## o que vamos fazer é trimestralizar a expectativa (média)

exp_ipca_aux = (
    exp_ipca
    # cria colunas de ano e mês para o agrupamento
    .assign(Year = lambda x: pd.to_datetime(x.Data).dt.year,
            Month = lambda x: pd.to_datetime(x.data).dt.month)
    #agrupa por mês e ano
    .groupby(by = ['Year','Month'])
    # calcula a média
    .agg({'Mediana':'mean'})
    .reset_index()
)

AttributeError: 'DataFrame' object has no attribute 'data'